# XGBoost

#### Import libraries

In [65]:
import matplotlib.pyplot as plt
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, accuracy_score, make_scorer, f1_score

#### Read data

In [66]:
df = pd.read_csv('../../../datasets/parte2/train_prepared.csv', na_filter=False)

#### Turn categorical features into numeric

In [67]:
replace_map = {'None':0, 'Low':1, 'Medium':2, 'High':3,'Very High':4}

df['Injeção na rede (kWh)'] = df['Injeção na rede (kWh)'].replace(replace_map).astype(int)

#### X and y arrays

In [68]:
X = df.drop(['Injeção na rede (kWh)'], axis=1)
y = df['Injeção na rede (kWh)']

#### Training 

##### Grid Search

Using GridSearchCV to find the best hyperparameters

In [69]:
param_grid = {
    'max_depth': [4], 
    'min_child_weight': [1, 2, 3],  
    'subsample': [0.5, 0.6, 0.7],  
    'colsample_bytree': [0.6, 0.8, 1], 
    'learning_rate': [0.01, 0.1],  
    'n_estimators': [100],  
}

model = XGBClassifier(random_state=2023)

#f1_scorer = make_scorer(f1_score, average='weighted')

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, refit=True, cv=5, verbose=1, n_jobs=-1, error_score="raise") # scoring='f1'
grid_search.fit(X, y)

Fitting 5 folds for each of 54 candidates, totalling 270 fits


c:\Users\gui\anaconda3\envs\DAA\lib\site-packages\xgboost\data.py:299: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
c:\Users\gui\anaconda3\envs\DAA\lib\site-packages\xgboost\data.py:301: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(dtype) and enable_categorical:
c:\Users\gui\anaconda3\envs\DAA\lib\site-packages\xgboost\data.py:332: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype)
c:\Users\gui\anaconda3\envs\DAA\lib\site-packages\xgboost\data.py:323: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is_int or is_bool or is_float or is_categori

GridSearchCV(cv=5, error_score='raise',
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=2023, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.6, 0.8, 1],
                         'learning_rate': [0.01, 0.1], 'max_depth': [4],
                         'min_child_weight': [1, 2, 3], 'n_estimators': [100],
                         'subsample': [0.5, 0.6, 0.7]},
             verbose=1)

Inspect the best parameters

In [70]:
grid_search.best_params_ # Melhor : {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 200}

{'colsample_bytree': 0.8,
 'learning_rate': 0.1,
 'max_depth': 4,
 'min_child_weight': 2,
 'n_estimators': 100,
 'subsample': 0.5}

In [71]:
grid_search.best_score_ # 0.0.8936697222898247 // 0.8936689393880897

0.8616576307015759

Get the best estimator

In [72]:
model = grid_search.best_estimator_

Get the predictions using the trained model

In [73]:
#predictions = model.predict(X)

##### Model

In [74]:
#model = XGBClassifier(random_state=2023, learning_rate=0.01, colsample_bytree=0.9, max_depth=4, n_estimators=300, subsample=0.7)

In [75]:
#model.fit(X, y)

#### Model Evaluation

In [76]:
#scores = cross_val_score(model, X, y, cv=10, n_jobs=-1)

In [77]:
#print(scores)

In [78]:
#print("Mean accuracy:", scores.mean())

In [79]:
df_test = pd.read_csv('../../../datasets/parte2/test_prepared.csv')
predictions_test = model.predict(df_test)

c:\Users\gui\anaconda3\envs\DAA\lib\site-packages\xgboost\data.py:299: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
c:\Users\gui\anaconda3\envs\DAA\lib\site-packages\xgboost\data.py:301: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(dtype) and enable_categorical:
c:\Users\gui\anaconda3\envs\DAA\lib\site-packages\xgboost\data.py:332: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype)
c:\Users\gui\anaconda3\envs\DAA\lib\site-packages\xgboost\data.py:323: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is_int or is_bool or is_float or is_categori

In [80]:
df_predictions = pd.DataFrame({'Result': predictions_test})
df_predictions['RowId'] = range(1, len(predictions_test) + 1)
df_predictions = df_predictions[['RowId', 'Result']]

replace_map = { 0:'None', 1:'Low', 2:'Medium', 3:'High', 4:'Very High'} 

df_predictions['Result'] = df_predictions['Result'].replace(replace_map)

df_predictions.to_csv('../../../datasets/parte2/kaggle-xgboost-holidays.csv', index=False)